<a href="https://colab.research.google.com/github/cwscx/makmore/blob/main/makemore_becoming_a_backprop_ninjia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# from google.colab import files
# uploaded = files.upload()

In [ ]:
words = open('names.txt', 'r').read().splitlines()

In [ ]:
len(words)

In [ ]:
stoi = {c:i for i, c in enumerate('.abcdefghijklmnopqrstuvwxyz')}
itos = {i:c for c, i in stoi.items()}

In [ ]:
# Build inputs.
block_size = 3

def build_dataset(words: list[str]):
  x: list[list[int]] = []
  y: list[int] = []
  for w in words:
    inputs = [0] * block_size
    output = None
    for c in w + '.':
      c_index = stoi[c]
      x.append(inputs)
      y.append(c_index)
      inpupts = inputs[1:] + [c_index]

  return torch.tensor(x), torch.tensor(y)

import random
random.seed(42)
random.shuffle(words)

n1 = int(len(words) * 0.8)
n2 = int(len(words) * 0.9)

training_x, training_y = build_dataset(words[:n1])
dev_x, dev_y = build_dataset(words[n1:n2])
test_x, test_y = build_dataset(words[n2:])

In [ ]:
def cmp(s, dt, t):
  ex = torch.all(dt == t.grad).item()
  app = torch.allclose(dt, t.grad)
  maxdiff = (dt - t.grad).abs().max().item()
  print(f'{s:15s} | exact: {str(ex):5s} | approximate: {str(app):5s} | maxdiff: {maxdiff}')

In [ ]:
n_embed = 10
n_hidden = 64
vocab_size = len(itos)

g = torch.Generator().manual_seed(2147483647)
C = torch.randn((vocab_size, 10), generator=g)
W1 = torch.randn((block_size * n_embed, n_hidden), generator=g) * (5/3) / ((block_size * n_embed)**0.5)
b1 = torch.randn((n_hidden), generator=g) * 0.1
W2 = torch.randn((n_hidden, vocab_size), generator=g) * 0.1
b2 = torch.randn((vocab_size), generator=g) * 0

bngain = torch.ones((1, n_hidden)) * 0.1 + 1.0
bnbias = torch.zeros((1, n_hidden)) * 0.1

parameters = [C, W1, b1, W2, b2, bngain, bnbias]

for p in parameters:
  p.requires_grad = True

In [ ]:
batch_size = 32
n = batch_size

ix = torch.randint(0, high=training_x.shape[0], size=(batch_size,), generator=g)

Xb = training_x[ix]
Yb = training_y[ix]

In [ ]:

emb = C[Xb]
embcat = emb.view(emb.shape[0], -1)

# Linear layer 1
hprebn = embcat @ W1 + b1 # pre activation

# BatchNorm layer
bnmeani = 1 / n * hprebn.sum(0, keepdim=True)
bndiff = hprebn - bnmeani
bndiff2 = bndiff**2
bnvar = 1 / (n - 1) * bndiff2.sum(0, keepdim=True)
bnvar_inv = (bnvar + 1e-5) ** -0.5
bnraw = bndiff * bnvar_inv
hpreact = bngain * bnraw + bnbias

# Non-linearity
h = torch.tanh(hpreact)

# Linear layer 2
logits = h @ W2 + b2

# Cross entropy
logit_maxes = logits.max(1, keepdim=True).values
norm_logits = logits - logit_maxes
counts = norm_logits.exp()
counts_sum = counts.sum(1, keepdim=True)
counts_sum_inv = counts_sum ** -1
probs = counts * counts_sum_inv
logprobs = probs.log()
# it should be
# log(y_target) - log(y_predict)
# log(1) - log(y_predict)
# 0 - log(y_predict)
# -log(y_predict)
loss = -logprobs[range(n), Yb].mean()

# bnmean_running = 0.999 * bnmean_running + 0.001 * batch_norm_mean
# bnstd_running = 0.999 * bnstd_running + 0.001 * batch_norm_std

for p in parameters:
  p.grad = None

for t in [logprobs, probs, counts, counts_sum, counts_sum_inv,
          norm_logits, logit_maxes, logits, h, hpreact, bnraw,
          bnvar_inv, bnvar, bndiff2, bndiff, hprebn, bnmeani,
          embcat, emb]:
  t.retain_grad()

loss.backward()

In [ ]:
print(emb.shape)
print(C.shape)
print(Xb.shape)

In [ ]:
# Excersise 1: backprop through the whole thing manually,
# backpropagating throug exactly all of the variables as
# they are defined in the forward pass above, one by one.

# --------Code start-----------
import math

dlogprobs = torch.zeros(logprobs.shape)
dlogprobs[range(n), Yb] = -1.0 / n

dprobs = 1 / probs * dlogprobs

# shape is different.
# c = a * b
# a[3x3] * b[3x1]
# a11*b1 a12*b1 a13*b1
# a21*b2 a22*b2 a23*b2
# a31*b3 a32*b3 a33*b3
dcounts_sum_inv = (counts * dprobs).sum(dim=1, keepdim=True)
dcounts_sum = - 1 / counts_sum**2 * dcounts_sum_inv

# count_sum_div also depend on counts. Need two calculation.
# b = sum(a)
# a11 a12 a13 --> b1
# a21 a22 a23 --> b2
# a31 a32 a33 --> b3
# db  / da = 1
dcounts = counts_sum_inv * dprobs + dcounts_sum

dnorm_logits = norm_logits.exp() * dcounts

# c = b - a
# b(32x27) a(32x1)
# b11-a1 b12-a1 b13-a1 ... b127-a1
dlogit_maxes = (-dnorm_logits).sum(dim=1, keepdim=True)

# max
# b = max(a1, a2, a3, ..., a27)
# only the one that is taken has grad 1, others are 0.
dlogits = dnorm_logits.clone() + F.one_hot(logits.max(1).indices, num_classes=logits.shape[1]) * dlogit_maxes

dh = dlogits @ W2.T
dW2 = torch.transpose(h, 0, 1) @ dlogits
db2 = dlogits.sum(0, keepdim=True)

# y = tanh(x)
# dy / dx = 1 - y**2
dhpreact = dh * (1 - h**2)

dbngain = (dhpreact * bnraw).sum(dim=0, keepdim=True)
dbnbias = dhpreact.sum(dim=0, keepdim=True)
dbnraw = dhpreact * bngain

dbnvar_inv = (dbnraw * bndiff).sum(dim=0, keepdim=True)
dbnvar = dbnvar_inv * -0.5 * (bnvar + 1e-5)**-1.5
dbndiff2 = (1.0 / (n-1)) * torch.ones_like(bndiff2) * dbnvar

dbndiff = 2.0 * bndiff * dbndiff2 + bnvar_inv * dbnraw

dbnmeani = (-dbndiff).sum(dim=0, keepdim=True)
dhprebn = dbndiff + torch.ones_like(hprebn) * (dbnmeani / n)

dembcat = dhprebn @ W1.T
dW1 = embcat.T @ dhprebn
db1 = dhprebn.sum(0)

demb = dembcat.view(emb.shape[0], emb.shape[1], emb.shape[2])
dC = torch.zeros_like(C)
for k in range(Xb.shape[0]):
  for j in range(Xb.shape[1]):
    ix = Xb[k ,j]
    dC[ix] += demb[k, j]
# --------Code end-------------


# --------Test start-----------
cmp('logprobs', dlogprobs, logprobs)
cmp('probs', dprobs, probs)
cmp('counts_sum_inv', dcounts_sum_inv, counts_sum_inv)
cmp('counts_sum', dcounts_sum, counts_sum)
cmp('counts', dcounts, counts)
cmp('norm_logits', dnorm_logits, norm_logits)
cmp('logit_maxes', dlogit_maxes, logit_maxes)
cmp('logits', dlogits, logits)
cmp('h', dh, h)
cmp('W2', dW2, W2)
cmp('b2', db2, b2)
cmp('hpreact', dhpreact, hpreact)
cmp('bngain', dbngain, bngain)
cmp('bnbias', dbnbias, bnbias)
cmp('bnraw', dbnraw, bnraw)
cmp('bnvar_inv', dbnvar_inv, bnvar_inv)
cmp('bnvar', dbnvar, bnvar)
cmp('bndiff2', dbndiff2, bndiff2)
cmp('bndiff', dbndiff, bndiff)
cmp('bnmeani', dbnmeani, bnmeani)
cmp('hprebn', dhprebn, hprebn)
cmp('embcat', dembcat, embcat)
cmp('W1', dW1, W1)
cmp('b1', db1, b1)
cmp('emb', demb, emb)
cmp('C', dC, C)
# --------Test end-------------

In [ ]:
# Exercise 2: backprop through cross entropy but all in one go
# to complete this challenge look at the mathmatical expression
# of the loss, take the derivative, simplify the expression, and
# just write it out.

# forward pass

# before:
# logit_maxes = logits.max(1, keepdim=True).values
# norm_logits = logits - logit_maxes
# counts = norm_logits.exp()
# counts_sum = counts.sum(1, keepdim=True)
# counts_sum_inv = counts_sum ** -1
# probs = counts * counts_sum_inv
# logprobs = probs.log()
# loss = -logprobs[range(n), Yb].mean()

# now
loss_fast = F.cross_entropy(logits, Yb)
print(loss_fast.item(), 'diff:', (loss_fast - loss).item())

In [ ]:
# backward pass

#------Code start--------
# loss = -log(p_y)
#      = -log(e^l_i / sum(e^l))
# dlogits = dloss / dl_i
#
# dloss / dl_i = p_i if not_match else 1 - p_i
dlogits = F.softmax(logits, dim=1)
dlogits[range(n), Yb] -= 1
dlogits /= n
#------Code end----------

cmp('logits', dlogits, logits) # 6e-9

In [ ]:
plt.figure(figsize=(3,3))
plt.imshow(dlogits.detach(), cmap="gray")

In [ ]:
# Exercise 3: backdrop through batchnorm but all in one go
# to complete the challenge look at the mathmatical expression
# of hte output of the batchnorm, the the derivative w.r.t. its
# input, simplify the expression, and just write it out.

# forward pass
# hprebn = embcat @ W1 + b1 # pre activation
# bnmeani = 1 / n * hprebn.sum(0, keepdim=True)
# bndiff = hprebn - bnmeani
# bndiff2 = bndiff**2
# bnvar = 1 / (n - 1) * bndiff2.sum(0, keepdim=True)
# bnvar_inv = (bnvar + 1e-5) ** -0.5
# bnraw = bndiff * bnvar_inv
# hpreact = bngain * bnraw + bnbias

# calculate dhprebn given dhpreact

#------Code start--------
dhprebn = bngain * bnvar_inv/n * (n*dhpreact - dhpreact.sum(0) - n/(n-1)*bnraw*(dhpreact*bnraw).sum(0))
#------Code end----------

cmp('hprebn', dhprebn, hprebn)